In [1]:
import os  # Взаимодействие с файловой системой
import sys # Доступ к некоторым переменным и функциям Python

PATH_TO_SOURCE = os.path.abspath(os.path.dirname(globals()['_dh'][0]))
PATH_TO_ROOT = os.path.join(PATH_TO_SOURCE, '..', '..', '..')

sys.path.insert(0, os.path.abspath(PATH_TO_ROOT))

In [2]:
import os
import pandas as pd
import gradio as gr
from oceanai.modules.lab.build import Run

def oceanai_initialization():
    out = False

    # Создание экземпляра класса
    _b5 = Run(lang='en',metadata=out)

    # Настройка ядра
    _b5.path_to_save_ = './models' # Директория для сохранения файла
    _b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

    corpus = 'fi'

    # Формирование аудиомоделей
    res_load_model_hc = _b5.load_audio_model_hc(out=out)
    res_load_model_nn = _b5.load_audio_model_nn(out=out)

    # Загрузка весов аудиомоделей
    url = _b5.weights_for_big5_['audio'][corpus]['hc']['sberdisk']
    res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url, out=out)

    url = _b5.weights_for_big5_['audio'][corpus]['nn']['sberdisk']
    res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url, out=out)

    # Формирование видеомоделей
    res_load_model_hc = _b5.load_video_model_hc(lang='en', out=out)
    res_load_model_deep_fe = _b5.load_video_model_deep_fe(out=out)
    res_load_model_nn = _b5.load_video_model_nn(out=out)

    # Загрузка весов видеомоделей
    url = _b5.weights_for_big5_['video'][corpus]['hc']['sberdisk']
    res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url, out=out)

    url = _b5.weights_for_big5_['video'][corpus]['fe']['sberdisk']
    res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url, out=out)

    url = _b5.weights_for_big5_['video'][corpus]['nn']['sberdisk']
    res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url, out=out)

    # Загрузка словаря с экспертными признаками (текстовая модальность)
    res_load_text_features = _b5.load_text_features(out=out)

    # Формирование текстовых моделей 
    res_setup_translation_model = _b5.setup_translation_model() # только для русского языка
    res_setup_translation_model = _b5.setup_bert_encoder(force_reload = False, out=out)
    res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus, out=out)
    res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus, out=out)

    # Загрузка весов текстовых моделей
    url = _b5.weights_for_big5_['text'][corpus]['hc']['sberdisk']
    res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url, out=out)

    url = _b5.weights_for_big5_['text'][corpus]['nn']['sberdisk']
    res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url, out=out)

    # Формирование модели для мультимодального объединения информации
    res_load_avt_model_b5 = _b5.load_avt_model_b5(out=out)

    # Загрузка весов модели для мультимодального объединения информации
    url = _b5.weights_for_big5_['avt'][corpus]['b5']['sberdisk']
    res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url, out=out)
    return _b5

def function_read_audio(files):
    out = False
    _b5.get_avt_predictions_gradio(paths = files, url_accuracy = '', accuracy=False,lang = 'en', out=out)

    return _b5.df_files_

demo = gr.Interface(
    function_read_audio,
    gr.File(file_count="multiple"),
    "dataframe"
)

if __name__ == "__main__":
    _b5 = oceanai_initialization()
    demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [1]:
import yaml
from typing import Dict, List, Tuple, Union, Generator

# Importing necessary components for the Gradio app


def load_practical_tasks_data(file_path: str) -> List:
    with open(file_path, "r") as file:
        return yaml.safe_load(file) or []

yaml_file_path = "/Users/dl/HF/OCEANAI/practical_tasks.yaml"
practical_tasks_data = load_practical_tasks_data(yaml_file_path)
practical_tasks_data

[{'task': 'Ranking potential candidates by professional responsibilities',
  'subtasks': ['Professional groups', 'Professional skills']},
 {'task': 'Forming effective work teams',
  'subtasks': ['Finding a suitable junior colleague',
   'Finding a suitable senior colleague']},
 {'task': 'Predicting consumer preferences for industrial goods',
  'subtasks': ['Car characteristics', 'Mobile device application categories']}]

In [6]:
def transform_practical_tasks_data(data: List) -> Dict:
    output_dict = {item['task']: item['subtasks'] for item in data}
    return output_dict

supported_practical_tasks = transform_practical_tasks_data(practical_tasks_data)
supported_practical_tasks

{'Ranking potential candidates by professional responsibilities': ['Professional groups',
  'Professional skills'],
 'Forming effective work teams': ['Finding a suitable junior colleague',
  'Finding a suitable senior colleague'],
 'Predicting consumer preferences for industrial goods': ['Car characteristics',
  'Mobile device application categories']}

In [23]:
radio_groups = {}
for value, label, choices, info, interactive, visible in [
    (
        None,
        "Practical tasks",
        list(map(str, supported_practical_tasks.keys())),
        "Choose a practical task",
        True,
        True,
    ),
    (
        None,
        "Practical subtasks",
        list(map(list, supported_practical_tasks.values())),
        "Choose a practical subtask",
        False,
        False,
    ),
]:
    print(label, choices)

Practical tasks ['Ranking potential candidates by professional responsibilities', 'Forming effective work teams', 'Predicting consumer preferences for industrial goods']
Practical subtasks [['Professional groups', 'Professional skills'], ['Finding a suitable junior colleague', 'Finding a suitable senior colleague'], ['Car characteristics', 'Mobile device application categories']]


In [25]:
first_practical_task = next(iter(supported_practical_tasks))
supported_practical_tasks[first_practical_task]

['Professional groups', 'Professional skills']

In [29]:
import gradio as gr

with gr.Blocks() as gradio_app:
    practical_tasks = gr.Radio(
        value='2',
        label="Practical tasks",
        choices=["1", "2", "3"],
        info="Choose a practical task",
        show_label=True,
        container=True,
        interactive=True,
        visible=True,
    )
    
    print(practical_tasks.value)

gradio_app.queue(api_open=False).launch(share=False)

1
Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
